## Project two: character-level language modeling in TensorFlow

### Preprocessing the dataset

In [1]:
! curl -O https://www.gutenberg.org/files/1268/1268-0.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1143k  100 1143k    0     0   124k      0  0:00:09  0:00:09 --:--:--  193k


In [2]:
import numpy as np

## Reading and preprocessing text
with open('1268-0.txt', 'r') as fp:
    text = fp.read()
    
start_indx = text.find('THE MYSTERIOUS ISLAND')
end_indx = text.find('End of the Project Gutenberg')
text = text[start_indx:end_indx]
char_set = set(text)
print('Total Length:', len(text))
print('Unique Characters:', len(char_set))

Total Length: 1130711
Unique Characters: 85


In [3]:
chars_sorted = sorted(char_set)
char2int = {ch:i for i, ch in enumerate(chars_sorted)}
char_array = np.array(chars_sorted)

text_encoded = np.array([char2int[ch] for ch in text], dtype=np.int32)
print('Text encoded shape:', text_encoded.shape)

print(text[:15], '== Encoding ==>', text_encoded[:15])
print(text_encoded[:15], '== Reverse ==>', ''.join(char_array[text_encoded[:15]]))

Text encoded shape: (1130711,)
THE MYSTERIOUS  == Encoding ==> [48 36 33  1 41 53 47 48 33 46 37 43 49 47  1]
[48 36 33  1 41 53 47 48 33 46 37 43 49 47  1] == Reverse ==> THE MYSTERIOUS 


Now, we will create a TensorFlow dataset from this array:

In [4]:
import tensorflow as tf

ds_text_encoded = tf.data.Dataset.from_tensor_slices(text_encoded)

for ex in ds_text_encoded.take(5):
    print('{} -> {}'.format(ex.numpy(), char_array[ex.numpy()]))

2023-08-18 17:30:26.634104: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-18 17:30:26.673864: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-18 17:30:26.674466: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-18 17:30:27.501169: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


48 -> T
36 -> H
33 -> E
1 ->  
41 -> M


2023-08-18 17:30:28.532374: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
seq_length = 40
chunk_size = 41

ds_chunks = ds_text_encoded.batch(chunk_size, drop_remainder=True)

In [6]:
## define the function for splitting x & y
def split_input_target(chunk):
    input_seq = chunk[:-1]
    target_seq = chunk[1:]
    return input_seq, target_seq

ds_sequences = ds_chunks.map(split_input_target)

In [7]:
## inspection:
for example in ds_sequences.take(2):
    print(' Input (x):', repr(''.join(char_array[example[0].numpy()])))
    print('Target (y):', repr(''.join(char_array[example[1].numpy()])))
    print()

 Input (x): 'THE MYSTERIOUS ISLAND ***\n\n\n\n\nTHE MYSTER'
Target (y): 'HE MYSTERIOUS ISLAND ***\n\n\n\n\nTHE MYSTERI'

 Input (x): 'OUS ISLAND\n\nby Jules Verne\n\n1874\n\n\n\n\nPAR'
Target (y): 'US ISLAND\n\nby Jules Verne\n\n1874\n\n\n\n\nPART'



In [8]:
# Batch size
BATCH_SIZE = 64
BUFFER_SIZE = 10000

tf.random.set_seed(1)
ds = ds_sequences.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)# drop_remainder=True)

ds

<_BatchDataset element_spec=(TensorSpec(shape=(None, 40), dtype=tf.int32, name=None), TensorSpec(shape=(None, 40), dtype=tf.int32, name=None))>

### Building a character-level RNN model

In [11]:
def build_model(vocab_size, embedding_dim, rnn_units):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

## Setting the training parameters
charset_size = len(char_array)
embedding_dim = 256
rnn_units = 512

tf.random.set_seed(1)

model = build_model(
    vocab_size = charset_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 256)         21760     
                                                                 
 lstm_2 (LSTM)               (None, None, 512)         1574912   
                                                                 
 dense_2 (Dense)             (None, None, 85)          43605     
                                                                 
Total params: 1640277 (6.26 MB)
Trainable params: 1640277 (6.26 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
model.compile(
    optimizer='adam', 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True
    ))

model.fit(ds, epochs=10)

Epoch 1/10
431/431 [==============================] - 158s 361ms/step - loss: 2.3474
Epoch 2/10
431/431 [==============================] - 158s 364ms/step - loss: 1.7612
Epoch 3/10
431/431 [==============================] - 157s 363ms/step - loss: 1.5547
Epoch 4/10
431/431 [==============================] - 158s 365ms/step - loss: 1.4364
Epoch 5/10
431/431 [==============================] - 158s 364ms/step - loss: 1.3614
Epoch 6/10
431/431 [==============================] - 157s 363ms/step - loss: 1.3093
Epoch 7/10
431/431 [==============================] - 160s 370ms/step - loss: 1.2694
Epoch 8/10
431/431 [==============================] - 161s 371ms/step - loss: 1.2380
Epoch 9/10
431/431 [==============================] - 161s 371ms/step - loss: 1.2118
Epoch 10/10
431/431 [==============================] - 161s 372ms/step - loss: 1.1890


### Evaluation phase: generating new text passages

In [13]:
tf.random.set_seed(1)

logits = [[1.0, 1.0, 1.0]]
print('Probabilities:', tf.math.softmax(logits).numpy()[0])

samples = tf.random.categorical(
    logits=logits, num_samples=10)
tf.print(samples.numpy())

Probabilities: [0.33333334 0.33333334 0.33333334]
array([[0, 0, 1, 2, 0, 0, 0, 0, 1, 0]])


In [14]:
tf.random.set_seed(1)

logits = [[1.0, 1.0, 3.0]]
print('Probabilities:', tf.math.softmax(logits).numpy()[0])

samples = tf.random.categorical(
    logits=logits, num_samples=10)
tf.print(samples.numpy())

Probabilities: [0.10650698 0.10650698 0.78698605]
array([[2, 0, 2, 2, 2, 0, 1, 2, 2, 0]])


In [17]:
def sample(model, starting_str, 
           len_generated_text=500, 
           max_input_length=40,
           scale_factor=1.0):
    encoded_input = [char2int[s] for s in starting_str]
    encoded_input = tf.reshape(encoded_input, (1, -1))

    generated_str = starting_str

    model.reset_states()
    for i in range(len_generated_text):
        logits = model(encoded_input)
        logits = tf.squeeze(logits, 0)

        scaled_logits = logits * scale_factor
        new_char_indx = tf.random.categorical(
            scaled_logits, num_samples=1)
        
        new_char_indx = tf.squeeze(new_char_indx)[-1].numpy()    

        generated_str += str(char_array[new_char_indx])
        
        new_char_indx = tf.expand_dims([new_char_indx], 0)
        encoded_input = tf.concat(
            [encoded_input, new_char_indx],
            axis=1)
        encoded_input = encoded_input[:, -max_input_length:]

    return generated_str

tf.random.set_seed(1)
print(sample(model, starting_str='The island'))

The island mastering a sign of rof.”

“Perhaps an east coast of them it?” exclaimed Pencroft. “Eane was no fire, and therefore, my did not far.”

The affarming. from
some channel would ever a long substance, and producing a
hearth were about the house. There the simple light of man.

“Vollow Harding,” said Pencroft, a few might
be
procement by the wowern
spoke of the stirnary, threatened so, ovem the right bank of the submited lava, now which commediately fall and their iron-containst of
which Neb
resport


* **Predictability vs. randomness**

In [15]:
logits = np.array([[1.0, 1.0, 3.0]])

print('Probabilities before scaling:        ', tf.math.softmax(logits).numpy()[0])

print('Probabilities after scaling with 0.5:', tf.math.softmax(0.5*logits).numpy()[0])

print('Probabilities after scaling with 0.1:', tf.math.softmax(0.1*logits).numpy()[0])

Probabilities before scaling:         [0.10650698 0.10650698 0.78698604]
Probabilities after scaling with 0.5: [0.21194156 0.21194156 0.57611688]
Probabilities after scaling with 0.1: [0.31042377 0.31042377 0.37915245]


In [18]:
tf.random.set_seed(1)
print(sample(model, starting_str='The island', 
             scale_factor=2.0))

The island of the common here and done of the castaways should be sufficient to continue that the corral, and the reporter was properly enough to discover the whole of the convicts, everything was such an entered the shore and the sea, and soon as the side of the lake, it was only seen that the engineer in the border of the birds had been able to stow that the pirates were therefore continued to an hour to be done but to force the engineer, and he had been able to say, the convicts were at the constructio


In [19]:
tf.random.set_seed(1)
print(sample(model, starting_str='The island', 
             scale_factor=0.5))

The island forgofach Easmatef irricsion? kitted
ir.
Of Juage!” -asks hisherintie?s. Tabren Neme,
fiisaneer,
is, bark, upon,’-smoke, rauniceablys. E5d 5 Yasly! thus?

Mlercal, jamary?” ertric/Chree, “Monerforr,”
retlees; und all,-faving a quartermasomits.
Cyrus Harding coulded-meg, makloy! coh diffftings, butnef rod, HU. SBecUik
51 chrestor :
basanown” prLpool
Flanquplusawortment,”

replight. SyB)
haledders Gideon Spiletu, whindelygithfuls, aboth me,’mucovid, ab)eretcmetting inquilists. A yindo,
if
thly. H


---